In [1]:
import os
import json
import ast
import csv
import pandas as pd
import numpy as np

In [2]:
def created_folder(path):
    try:
        os.mkdir(path)
        return True
    except:
        return False
# count
def coordinate(x,y,w,h,origin_width,origin_height):
    x,y,w,h = float(x),float(y),float(w),float(h)
    x = x*0.01*origin_width
    y = y*0.01*origin_height
    w = w*0.01*origin_width
    h = h*0.01*origin_height
    return y,x,y+h,x+w

def create_new_dict(result):
    ans_old={}
    ans = []
    for info_bb in result:
        # info_bb_new = {}
        # value = {}
        # value['x'] = info_bb['value']['x']
        # value['y'] = info_bb['value']['y']
        # value['width'] = info_bb['value']['width']
        # value['height'] = info_bb['value']['height']
        # value['rectanglelabels'] = info_bb['value']['rectanglelabels']
        origin_width = info_bb['original_width']
        origin_height = info_bb['original_height']
        y1,x1,y2,x2 = coordinate(info_bb['value']['x'],info_bb['value']['y'],info_bb['value']['width'],info_bb['value']['height'],origin_width,origin_height)
        if info_bb['value']['rectanglelabels'][0] not in ans_old:
            ans_old[info_bb['value']['rectanglelabels'][0]] = [[y1,x1,y2,x2]]
        else:
            ans_old[info_bb['value']['rectanglelabels'][0]].append([y1,x1,y2,x2])
    return ans_old
import shutil
def remove_file_and_folder(path):
    shutil.rmtree(path)
            
def remove_file(path):
    os.remove(path)
created_folder('./trust_csv/')
remove_file_and_folder('./trust_csv/')
created_folder('./trust_csv/')
for file in os.listdir('./json'):
    file_json = open('./json/'+file)
    folder = file.split('.')[0].split('-')[1]
    print(folder)
    created_folder('./trust_csv/'+folder)
    data = json.load(file_json)
    # print(data)
    # print(data[0]['annotations'][0]['result'])
    header = ['frame_x','total','bounding_box']
    for i in range(len(data)):
        frame_name = data[i]['file_upload'].split('.')[0].split('-')[2]
        # print(frame_name)
        result = data[i]['annotations'][0]['result']
        # if int(frame_name) == 42:
            # print(result)
        if os.path.isfile('./trust_csv/'+folder+'/'+data[i]['file_upload'].split('.')[0].split('-')[1]+'.csv')==False:
            with open('./trust_csv/'+folder+'/'+data[i]['file_upload'].split('.')[0].split('-')[1]+'.csv','w') as f:
                write = csv.writer(f)
                write.writerow(header)
                f.close()

        with open('./trust_csv/'+folder+'/'+data[i]['file_upload'].split('.')[0].split('-')[1]+'.csv','a') as f:
            write = csv.writer(f)
            write.writerow([frame_name,len(result),create_new_dict(result)])
            f.close()

1
2
3
4


In [6]:
def sort_file_by_frame(path):
    for file in os.listdir(path):
        df = pd.read_csv(path+file) 
        df.astype({'frame_x':'int32'}).dtypes
        df.sort_values(by=['frame_x'],inplace=True)     
        remove_file(path+file)
        df.to_csv(path+file,index=False)

sort_file_by_frame('./trust_csv/4/')